In [1]:
import numpy as np
import pandas as pd
from scipy import stats

np.random.seed(1111)

In [2]:
df = pd.read_csv('./ch11_potato.csv')
df

,무게
0,122.02
1,131.73
2,130.60
3,131.82
4,132.05
5,126.12
6,124.43
7,132.89
8,122.79
9,129.95


In [3]:
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
s_mean = np.mean(sample)
s_mean

128.4507142857143

### 11.1 통계적 가설검정

- $(\overline{X}-130)/\sqrt\frac{9}{14}<z_{0.95}$이면 귀무가설을 기각
- $(\overline{X}-130)/\sqrt\frac{9}{14}\geq z_{0.95}$이면 귀무가설을 채택

In [7]:
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [9]:
# 단측검정
rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [10]:
rv.cdf(z) # p-value

0.026661319523126635

In [11]:
# 양측검정
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

In [12]:
rv.cdf(z)*2

0.05332263904625327

#### 가설검정의 두 가지 오류
- 제1종 오류: 귀무가설이 옳을 때, 귀무가설을 기각하는 오류
 - 위험률($\alpha$): 제1종 오류를 범하는 확률, 유의수준과 일치
- 제2종 오류: 대립가설이 옳을 때, 귀무가설을 채택하는 오류
 - 검정력($1-\beta$): $\beta$는 모집단의 정보에 의존하므로 제어할 수 없음

In [13]:
rv = stats.norm(130, 3)

In [14]:
# 제1종 오류를 범한 비율

c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_-130) / np.sqrt(9/14)
    if z < c:
        cnt += 1

cnt/n_samples

0.0528

In [17]:
rv = stats.norm(128, 3)

In [19]:
# 제2종 오류를 범한 비율

c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_-130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
cnt / n_samples

0.1969

### 11.2 기본적인 가설검정

#### 정규분포의 모평균에 대한 검정: 모분산을 알고 있는 경우
$X_1,X_2,\dots,X_n\stackrel{iid}{\sim}N(\mu,\sigma^2)$이라고 하자. 이때 모평균 $\mu$에 관한 유의수준 $\alpha$의 양측검정
- 귀무가설: $\mu=\mu_0$
- 대립가설: $\mu\neq\mu_0$

은, 검정통계량으로 $Z=(\overline{X}-\mu_0)/\sqrt\frac{\sigma^2}{n}$을 사용하여

$\begin{cases}Z<z_{1-\alpha/2}\;또는\;z_{\alpha/2}<Z라면,\;귀무가설을\;기각\\
z_{1-\alpha/2}\leq Z\leq z_{\alpha/2}라면,\;귀무가설을\;채택\end{cases}$

으로 수행된다.

In [20]:
# 양측검정

def pmean_test(sample, mean0, p_var, alpha=0.05):
    s_mean = np.mean(sample)
    n = len(sample)
    rv = stats.norm()
    interval = rv.interval(1-alpha)
    
    z = (s_mean-mean0) / np.sqrt(p_var/n)
    if interval[0] <= z <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
        
    if z < 0:
        p = rv.cdf(z) * 2
    else:
        p = (1-rv.cdf(z)) * 2
    print(f'p값은 {p:.3f}')

In [21]:
pmean_test(sample, 130, 9)

귀무가설을 채택
p값은 0.053


#### 정규분포의 모분산에 대한 검정
$X_1,X_2,\dots,X_n\stackrel{iid}{\sim}N(\mu,\sigma^2)$이라고 하자. 이때 모분산 $\sigma^2$에 관한 유의수준 $\alpha$의 양측검정
- 귀무가설: $\sigma^2=\sigma_0^2$
- 대립가설: $\sigma^2\neq\sigma_0^2$

은, 검정통계량으로 $Y=\frac{(n-1)s^2}{\sigma_0^2}$을 사용하여

$\begin{cases}Y<\chi_{1-\alpha/2}^2(n-1)\;또는\;\chi_{\alpha/2}^2(n-1)<Y라면,\;귀무가설을\;기각\\
\chi_{1-\alpha/2}^2(n-1)\leq Y\leq \chi_{\alpha/2}^2(n-1)라면,\;귀무가설을\;채택\end{cases}$

으로 수행된다.

In [22]:
def pvar_test(sample, var0, alpha=0.05):
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.chi2(df=n-1)
    interval = rv.interval(1-alpha)
    
    y = (n-1)*u_var / var0
    if interval[0] <= y <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
    if y < rv.isf(0.5):
        p = rv.cdf(y)*2
    else:
        p = (1-rv.cdf(y))*2
    print(f'p값은 {p:.3f}')

In [23]:
pvar_test(sample, 9)

귀무가설을 채택
p값은 0.085


#### 정규분포의 모평균에 대한 검정 : 모분산을 모르는 경우
$X_1,X_2,\dots,X_n\stackrel{iid}{\sim}N(\mu,\sigma^2)$이라고 하자. 이때 모평균 $\mu$에 관한 유의수준 $\alpha$의 양측검정
- 귀무가설: $\mu=\mu_0$
- 대립가설: $\mu\neq\mu_0$

은, 검정통계량으로 $t=(\overline{X}-\mu_0)/\sqrt\frac{s^2}{n}$을 사용하여

$\begin{cases}t<t_{1-\alpha/2}\;또는\;t_{\alpha/2}<t라면,\;귀무가설을\;기각\\
t_{1-\alpha/2}\leq t\leq t_{\alpha/2}라면,\;귀무가설을\;채택\end{cases}$

으로 수행된다.

In [26]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)
    
    t = (s_mean-mean0) / np.sqrt(u_var/n)
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')
    if t < 0:
        p = rv.cdf(t)*2
    else:
        p = (1-rv.cdf(t))*2
    print(f'p값은 {p:.3f}')

In [27]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


In [28]:
t, p = stats.ttest_1samp(sample, 130)
t, p

(-1.4551960206404198, 0.16933464230414275)

### 11.3 2표본 문제에 관한 가설검정

#### 대응비교 t 검정
- paired t-test: 대응하는 데이터가 있고, 데이터 차이에 정규분포를 가정할 수 있는 경우의 평균값 차이에 대한 검정

In [69]:
training_rel = pd.read_csv('./ch11_training_rel.csv')
print(training_rel.shape)
training_rel.head()

(20, 2)


,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84


- 귀무가설: $\mu_{after}-\mu_{before}=0$
- 대립가설: $\mu_{after}-\mu_{before}\neq 0$

In [47]:
training_rel['차'] = training_rel['후'] - training_rel['전']
training_rel.head()

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25


- 귀무가설: $\mu_{diff}=0$
- 대립가설: $\mu_{diff}\neq0$

In [31]:
t, p = stats.ttest_1samp(training_rel['차'], 0)
p

0.04004419061842953

In [33]:
t, p = stats.ttest_rel(training_rel['후'], training_rel['전'])
p

0.04004419061842953

#### 독립비교 t 검정
- independent t-test: 대응하는 데이터가 없고 독립된 2표본 모집단에 정규분포를 가정할 수 있는 경우 평균값의 차이에 대한 검정

In [34]:
training_ind = pd.read_csv('./ch11_training_ind.csv')
print(training_ind.shape)
training_ind.head()

(20, 2)


,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


- 귀무가설: $\mu_1-\mu_2=0$
- 대립가설: $\mu_1-\mu_2\neq 0$

- A학급 점수: $X_1,X_2,\dots,X_{n_1}\stackrel{iid}{\sim}N(\mu_1,\sigma_1^2)$
- B학급 점수: $Y_1,Y_2,\dots,Y_{n_1}\stackrel{iid}{\sim}N(\mu_2,\sigma_2^2)$

$t=\frac{(\overline{X}-\overline{Y})-(\mu_1-\mu_2)}{\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}}\qquad$
$df=\frac{\left(\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}\right)^2}{\frac{s_1^4}{n_1^2(n_1-1)}+\frac{s_2^4}{n_2^2(n_2-1)}}$

In [35]:
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var=False)
p

0.08695731107259361

#### 윌콕슨의 부호순위검정
- Wilcoxon signed-rank test: 대응표본에서 차이에 정규분포를 가정할 수 없는 경우, 중앙값의 차이에 대한 검정

In [50]:
toy_df = training_rel[:6].copy()
diff = toy_df['차']
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [54]:
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [55]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus 

# r_minus와 r_plus 중 더 작은 값이 검정통계량이 됨
# 검정통계량이 임계값보다 작은 경우 귀무가설이 기각되는 단측검정 수행

(8, 13)

In [59]:
toy_df['후'] = toy_df['전']+np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)

toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [61]:
r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus
# 차이에 편차가 있으면 검정통계량이 작아짐

(0, 21)

In [62]:
toy_df['후'] = toy_df['전']+[1,-2,-3,4,5,-6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6


In [63]:
r_minus = np.sum((diff < 0)*rank)
r_plus = np.sum((diff > 0)*rank)

r_minus, r_plus

(11, 10)

- 차이에 편향이 있을 수록 $W^+$와 $W^-$에 편향이 생기고, 검정통계량은 작은 값이 됨.
- 이러한 이론레 따라 검정통계량이 임계값보다 작으면 중앙값에 차이가 있다고 주장

In [70]:
w, p = stats.wilcoxon(training_rel['전'], training_rel['후'])
p

0.03623390197753906

In [76]:
w, p = stats.wilcoxon(training_rel['전'] - training_rel['후'])

p

0.03623390197753906

- 모집단이 정규분포를 따르고 있는 경우에도 윌콕슨 부호순위검정을 사용할 수 있지만, 대응비교 t검정의 검정력이 크다.

In [75]:
n = 10000
diffs = np.round(stats.norm(3,4).rvs(size=(n,20)))

In [77]:
cnt = 0
alpha = 0.05
for diff in diffs:
    t, p = stats.ttest_1samp(diff,0)
    if p < alpha:
        cnt += 1
cnt/n

0.8832

In [78]:
cnt = 0
alpha = 0.05
for diff in diffs:
    w, p = stats.wilcoxon(diff)
    if p < alpha:
        cnt += 1
cnt/n

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


0.8717

#### 만-위트니의 U검정
- Maan-Whitney rank test: 대응되는 데이터가 없는 2표본 모집단에 정규분포를 가정할 수 없는 경우, 중앙값의 차이에 대한 검정

In [79]:
training_ind = pd.read_csv('./ch11_training_ind.csv')
toy_df = training_ind[:5].copy()
toy_df

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


In [83]:
rank = stats.rankdata(np.concatenate([toy_df['A'], toy_df['B']]))
rank_df = pd.DataFrame({'A':rank[:5], 'B':rank[5:10].astype(int)})
rank_df

,A,B
0,3.0,5
1,6.0,8
2,1.0,9
3,10.0,4
4,2.0,7


In [84]:
n1 = len(rank_df['A'])
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

7.0

In [85]:
# A에 높은 순위가 모여있는 경우
rank_df = pd.DataFrame(np.arange(1, 11).reshape(2, 5).T, columns=['A', 'B'])
rank_df

,A,B
0,1,6
1,2,7
2,3,8
3,4,9
4,5,10


In [86]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

0.0

In [91]:
# A에 낮은 순위가 모여있는 경우
rank_df = pd.DataFrame(np.arange(1,11).reshape(2,5)[::-1].T, columns=['A','B'])
rank_df

,A,B
0,6,1
1,7,2
2,8,3
3,9,4
4,10,5


In [93]:
u = rank_df['A'].sum() - (n1*(n1+1))/2
u

25.0

In [96]:
u, p = stats.mannwhitneyu(training_ind['A'], training_ind['B'], alternative='two-sided')
p

0.05948611166127324

#### 카이제곱 검정

In [97]:
ad_df = pd.read_csv('./ch11_ad.csv')
n = len(ad_df)
print(n)
ad_df.head()

1000


,광고,구입
0,B,하지 않았다
1,B,하지 않았다
2,A,했다
3,A,했다
4,B,하지 않았다


In [98]:
ad_cross = pd.crosstab(ad_df['광고'], ad_df['구입'])
ad_cross

구입,하지 않았다,했다
광고,,
A,351,49
B,549,51


In [99]:
ad_cross['했다'] / (ad_cross['했다'] + ad_cross['하지 않았다'])

광고
A    0.1225
B    0.0850
dtype: float64

In [100]:
n_not, n_yes = ad_cross.sum()
n_not, n_yes

(900, 100)

In [102]:
n_adA, n_adB = ad_cross.sum(axis=1)
n_adA, n_adB

(400, 600)

In [104]:
ad_ef = pd.DataFrame({'했다':[n_adA*n_yes/n, n_adB*n_yes/n],
                      '하지 않았다':[n_adA*n_not/n, n_adB*n_not/n]},
                    index = ['A', 'B'])
ad_ef

,했다,하지 않았다
A,40.0,360.0
B,60.0,540.0


$Y=\sum_i\sum_j\frac{(O_{ij}-E_{ij})^2}{E_{ij}}$

In [105]:
y = ((ad_cross-ad_ef)**2/ad_ef).sum().sum()
y

3.75

In [109]:
rv = stats.chi2(1)
1 - rv.cdf(y)

0.052807511416113395

In [110]:
chi2, p, dof, ef = stats.chi2_contingency(ad_cross, correction=False)
chi2, p, dof

(3.75, 0.052807511416113395, 1)

In [111]:
ef

array([[360.,  40.],
       [540.,  60.]])